![CAO](https://upload.wikimedia.org/wikipedia/commons/5/51/Central_Applications_Office.png)

# CAO Points Analysis

http://www.cao.ie/index.php?page=points&p=2021
***

In [1]:
# Regular expressions.
import re

# Convenient HTTP requests.
import requests as rq

# Import Pandas
import pandas as pd

# Dates and times.
import datetime as dt

#import csv
import csv

#import camelot
import camelot

# For downloading.
import urllib.request as urlrq

In [2]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Have a quick peek. 200 means OK.
resp

<Response [200]>

In [3]:
# Get the current date and time.
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
# Create a file path for the original data.
path2021 = 'data/cao2021_' + nowstr + '.html'

In [5]:

# The server uses the wrong encoding, fix it.
original_encoding = resp.encoding

# Change to cp1252.
resp.encoding = 'cp1252'

In [6]:
# Save the original html file.
with open(path2021, 'w') as f:
    f.write(resp.text)

Compile the regular expression for matching lines. r' means raw string notation. [01] This regular expression means that we are looking for 2
characer set of upper case letters from A-Z. The {}'s denote how many
occurnaces of the character set we are looking for. This is followed by 3
groups. Groups allow us to match several different patterns. The first group
is any character except a newline (.) with 0 or more repetitions (*). The
second group is 3 digits (\d{3}). The third group is a literal * which is
optional. The ? denotes that it is optional.

In [7]:
re_course = re.compile(r'([A-Z]{2}\d{3})  (.*)')

### Loop through the lines of the response
***

In [8]:
# The file path for the csv file.
path2021 = 'data/cao2021_csv_' + nowstr + '.csv'

In [9]:
# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021, 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


In [10]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [11]:
df2021

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


### Cleaning up the Data

In [12]:
df2021.columns = ['code', 'title', 'points_r1', 'points_r2']
df2021

,code,title,points_r1,points_r2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


In [13]:
df2021.iloc[842]

code                                                     MH801
title        Early Childhood - Teaching and Learning (part-...
points_r1                                                  AQA
points_r2                                                  AQA
Name: 842, dtype: object

In [14]:
#  [11]

df2021['points_r1'].replace('[a-z.#*+A-Z]', '', regex = True, inplace = True)
df2021['points_r2'].replace('[a-z.#*+A-Z]', '', regex = True, inplace = True)
df2021

,code,title,points_r1,points_r2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


In [15]:
df2021.iloc[842]

code                                                     MH801
title        Early Childhood - Teaching and Learning (part-...
points_r1                                                     
points_r2                                                     
Name: 842, dtype: object

### Data Types

In [16]:
# Check the data types [10]
df2021.dtypes

code         object
title        object
points_r1    object
points_r2    object
dtype: object

In [17]:
# change the points columns from object to float [09]

df2021['points_r1'] = pd.to_numeric(df2021['points_r1'])
df2021['points_r2'] = pd.to_numeric(df2021['points_r2'])
df2021

,code,title,points_r1,points_r2
0,AL801,Software Design for Virtual Reality and Gaming,300.0,NaN
1,AL802,Software Design in Artificial Intelligence for...,313.0,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350.0,NaN
3,AL805,Computer Engineering for Network Infrastructure,321.0,NaN
4,AL810,Quantity Surveying,328.0,NaN
...,...,...,...,...
944,WD211,Creative Computing,270.0,NaN
945,WD212,Recreation and Sport Management,262.0,NaN
946,WD230,Mechanical and Manufacturing Engineering,230.0,230.0
947,WD231,Early Childhood Care and Education,266.0,NaN


In [18]:
df2021.dtypes

code          object
title         object
points_r1    float64
points_r2    float64
dtype: object

### Save clean data

In [19]:
# Create a file path for the pandas data
path2021clean = 'data/cao2021_clean_csv_' + nowstr + '.csv'

In [20]:
# Save pandas data frame to disk
df2021.to_csv(path2021clean) 

<br>

## 2020 Points

https://www.cao.ie/index.php?page=points&p=2020
***

In [21]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

**Save Original File**

In [22]:
# Create a file path for the original data.
pathxlsx = 'data/cao2020_' + nowstr + '.xlsx'

In [23]:
urlrq.urlretrieve(url2020, pathxlsx)

('data/cao2020_20211223_124042.xlsx',
 <http.client.HTTPMessage at 0x1bdffdeab20>)

## Load Spreadsheet using pandas

In [24]:
# Download and parse the excel spreadsheet.
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows=10)

In [25]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Spot check a random row.
df2020.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [27]:
# Spot check the last row.
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [28]:
# Create a file path for the pandas data.
path = 'data/cao2020_' + nowstr + '.csv'

In [29]:
# Save pandas data frame to disk.
df2020.to_csv(path)

In [30]:
# Get rid of the level 7 courses [12]

df2020 = df2020.loc[df2020["LEVEL"] > 7]
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Arts,Arts (options),WD200,AQA,NaN,AQA,NaN,AQA,NaN,336,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df2020.columns

Index(['CATEGORY (i.e.ISCED description)', 'COURSE TITLE', 'COURSE CODE2',
       'R1 POINTS', 'R1 Random *', 'R2 POINTS', 'R2 Random*', 'EOS',
       'EOS Random *', 'EOS Mid-point', 'LEVEL', 'HEI', 'Test/Interview #',
       'avp', 'v', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5',
       'Column6', 'Column7', 'Column8'],
      dtype='object')

In [32]:
pd.Series(df2020.columns)

0     CATEGORY (i.e.ISCED description)
1                         COURSE TITLE
2                         COURSE CODE2
3                            R1 POINTS
4                          R1 Random *
5                            R2 POINTS
6                           R2 Random*
7                                  EOS
8                         EOS Random *
9                        EOS Mid-point
10                               LEVEL
11                                 HEI
12                    Test/Interview #
13                                 avp
14                                   v
15                             Column1
16                             Column2
17                             Column3
18                             Column4
19                             Column5
20                             Column6
21                             Column7
22                             Column8
dtype: object

### Cleaning up the Data

In [33]:
# Select only Columns of Interest [11]

df2020 = df2020.loc[:, ['COURSE CODE2', 'COURSE TITLE', 'R1 POINTS', 'R2 POINTS', 'EOS', 'EOS Mid-point']]

# Remove all non-numeric characters that appeared in the Points columns
df2020['R1 POINTS'].replace('[a-z.#*+A-Z]', '',regex = True, inplace = True)
df2020['R2 POINTS'].replace('[a-z.#*+A-Z]', '',regex = True, inplace = True)
df2020['EOS Mid-point'].replace('[a-z.#*+A-Z]', '',regex = True, inplace = True)
df2020['EOS'].replace('[a-z.#*+A-Z]', '', regex = True, inplace = True)

# Rename the columns
df2020.columns = ['code', 'title', 'points_r1', 'points_r2', 'eos_points', 'mid-point']

df2020

,code,title,points_r1,points_r2,eos_points,mid-point
0,AC120,International Business,209,NaN,209,280
1,AC137,Liberal Arts,252,NaN,252,270
2,AD101,"First Year Art & Design (Common Entry,portfolio)",,NaN,,
3,AD102,Graphic Design and Moving Image Design (portfo...,,NaN,,
4,AD103,Textile & Surface Design and Jewellery & Objec...,,NaN,,
...,...,...,...,...,...,...
1455,WD200,Arts (options),,,,336
1460,WD210,Software Systems Development,279,NaN,279,337
1461,WD211,Creative Computing,271,NaN,271,318
1462,WD212,Recreation and Sport Management,270,NaN,270,349


### Data Types

In [34]:
# Check the data types [10]
df2020.dtypes

code          object
title         object
points_r1     object
points_r2     object
eos_points    object
mid-point     object
dtype: object

In [35]:
# change the points columns from object to float [09]

df2020['points_r1'] = pd.to_numeric(df2020['points_r1'])
df2020['points_r2'] = pd.to_numeric(df2020['points_r2'])
df2020['eos_points'] = pd.to_numeric(df2020['eos_points'])
df2020['mid-point'] = pd.to_numeric(df2020['mid-point'])
df2020

,code,title,points_r1,points_r2,eos_points,mid-point
0,AC120,International Business,209.0,NaN,209.0,280.0
1,AC137,Liberal Arts,252.0,NaN,252.0,270.0
2,AD101,"First Year Art & Design (Common Entry,portfolio)",NaN,NaN,NaN,NaN
3,AD102,Graphic Design and Moving Image Design (portfo...,NaN,NaN,NaN,NaN
4,AD103,Textile & Surface Design and Jewellery & Objec...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1455,WD200,Arts (options),NaN,NaN,NaN,336.0
1460,WD210,Software Systems Development,279.0,NaN,279.0,337.0
1461,WD211,Creative Computing,271.0,NaN,271.0,318.0
1462,WD212,Recreation and Sport Management,270.0,NaN,270.0,349.0


In [37]:
df2020.dtypes

code           object
title          object
points_r1     float64
points_r2     float64
eos_points    float64
mid-point     float64
dtype: object

### Save Clean Data

In [38]:
# Create a file path for the pandas data
path2020clean = 'data/cao2020_clean_csv_' + nowstr + '.csv'

In [39]:
# Save pandas data frame to disk
df2020.to_csv(path2020clean) 

<br>

## 2019 Points
https://www.cao.ie/index.php?page=points&p=2019
***

**_Step to reproduce_**
<br>

1. Download and Install [Ghostscript](https://www.ghostscript.com/releases/gsdnld.html). Choose public license 64 bit.
2. Install [Camelot](https://camelot-py.readthedocs.io/en/master/) using the command ``conda install -c conda-forge camelot-py`` in cmder.
3. Import the Camelot package using the command ``import camelot`` as seen in the import cell at the top of this project.
4. Create a **[TableList](https://camelot-py.readthedocs.io/en/master/api.html#camelot.core.TableList)** object called ``tables``, which is a list of Table objects using the file path of the 2019 data as seen below.
5. Seince there are 18 pages in this PDF, Set ``the read_pdf`` parameter as a range from 1-18 

In [ ]:
tables = camelot.read_pdf('data/cao2019_20211104_143030.pdf', pages = '1-18')

In [ ]:
tables

Above, we can see that the tables object has 18 tables, since n=18.

In [ ]:
type(tables)

To access a table by using its index. We can select a table passing the index.

In [ ]:
tables[0]

The above is the first table on the PDF file. We can also see the shape of the table, 4 rows and 44 columns. The next thing to do is to make an object for each table index below so that we can use pandas to concatonate all of the tables.

In [ ]:
table_1 = tables[0]
table_2 = tables[1]
table_3 = tables[2]
table_4 = tables[3]
table_5 = tables[4]
table_6 = tables[5]
table_7 = tables[6]
table_8 = tables[7]
table_9 = tables[8]
table_10 = tables[9]
table_11 = tables[10]
table_12 = tables[11]
table_13 = tables[12]
table_14 = tables[13]
table_15 = tables[14]
table_16 = tables[15]
table_17 = tables[16]
table_18 = tables[17]

Let's take a look at the first and last tables using the df function in Camelot.

In [ ]:
table_1.df

In [ ]:
table_18.df

We need to drop the rows that are not course names (Name of the college and the first row in table_1 which is just the headers which we will write ourselves later. To do this, take a look at each table and find the indexes that need to be dropped.for table_1 we need to drop indexes 0, 1 and 29. I will just show how the first table because the process just needs to be repeated for the rest of the tables.

In [ ]:
# Dropped the indexes that we do not need
table_1.df.drop([0, 1, 29])

Once we know which rows to drop we then need to make the drop operations that we just performed permanent since these operations are not inplace. To do this we need to perform the operation for each table and then assign the operations back to the original variables as seen below. We could use the ``inplace = True`` option of the function, but this example it is neater to assign the operations back to the original variable i.e. the tables.

In [ ]:
# Assign the drop operations back to the original variables.
table_1 = table_1.df.drop([0, 1, 29])
table_2 = table_2.df.drop([19, 23])
table_3 = table_3.df.drop([12])
table_4 = table_4.df.drop([20, 23, 25, 32])
table_5 = table_5.df.drop([1])
table_6 = table_6.df.drop([12])
table_7 = table_7.df.drop([27, 45])
table_8 = table_8.df.drop([9, 13, 32])
table_9 = table_9.df.drop([2, 13, 22, 27])
table_10 = table_10.df.drop([30])
table_11 = table_11.df.drop([17, 40])
table_12 = table_12.df.drop([33])
table_13 = table_13.df.drop([45])
table_14 = table_14.df.drop([10, 24, 22])
table_15 = table_15.df.drop([21])
table_16 = table_16.df.drop([8, 11, 47])
table_17 = table_17.df.drop([0, 33])
table_18 = table_18.df.drop([4])

We can now make an dataframe using the concatonate function in pandas.

In [ ]:
df2019 = pd.concat([table_1, table_2, table_3, table_4, table_5,
                    table_6, table_7, table_8, table_9, table_10,
                    table_11, table_12, table_13, table_14, table_15,
                    table_16, table_17, table_18], ignore_index = True)

In [ ]:
df2019.columns = ['code','title', 'points_r1', 'points_r2']
df2019

In [ ]:
df2019['points_r2'].dtype

## Concatonate and Join

In [ ]:
courses2021 = df2021[['code', 'title']]
courses2021

In [ ]:
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
courses2020.columns = ['code', 'title']
courses2020

In [ ]:
courses2019 = df2019[['code', 'title']]
courses2019

In [ ]:
allcourses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)
allcourses

In [ ]:
allcourses.sort_values('code')

In [ ]:
allcourses.loc[175]['title']

In [ ]:
allcourses.loc[949]['title']

In [ ]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

In [ ]:
# Returns a copy of the data frame with duplciates removed.
allcourses.drop_duplicates()

In [ ]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated(subset=['code'])]

In [ ]:
# Returns a copy of the data frame with duplciates removed - based only on code.
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [ ]:
allcourses

## Join the Points

In [ ]:
# Set the index to the code column.
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

In [ ]:
# Set the index to the code column.
allcourses.set_index('code', inplace=True)

In [ ]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

In [ ]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

In [ ]:
# Set the index to the code column.
df2020_r1.set_index('code', inplace=True)
df2020_r1

In [ ]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses

In [ ]:
df2019_r1 = df2019[['code', 'points_r1']]
df2019_r1.columns = ['code', 'points_r1_2019']
df2019_r1

In [ ]:
# Set the index to the code column.
df2019_r1.set_index('code', inplace=True)
df2019_r1

In [ ]:
# Join 2019 points to allcourses.
allcourses = allcourses.join(df2019_r1)
allcourses

In [ ]:
allcourses.loc['WD148']

In [ ]:
allcourses.iloc[1213]

In [ ]:
allcourses.sort_values('code')

## References
***

[01][The official Python Regular Expressions Documentation](https://docs.python.org/3/library/re.html)<br>
[02][Camelot: PDF Table Extraction for Humans](https://camelot-py.readthedocs.io/en/master/)<br>
[03][How to Extract tabular data from PDF document using Camelot in Python](https://www.analyticsvidhya.com/blog/2020/08/how-to-extract-tabular-data-from-pdf-document-using-camelot-in-python/)<br>
[04][Camelot - Quickstart](https://camelot-py.readthedocs.io/en/master/user/quickstart.html)<br>
[05][Camelot - API Documentation](https://camelot-py.readthedocs.io/en/master/api.html)<br>
[06][Ghostscript](https://www.ghostscript.com/releases/gsdnld.html)<br>
[07][pandas.concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)<br>
[08][Stackoverflow - Merging two or more continuous tables from a list in Python](https://stackoverflow.com/questions/60082511/merging-two-or-more-continuous-tables-from-a-list-in-python)<br>
[09][Data to Fish - How to Convert Strings to Integers in Pandas DataFrame](https://datatofish.com/string-to-integer-dataframe/)<br>
[10][pandas - pandas.DataFrame.dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html)<br>
[11][GeeksforGeeks - Python | Pandas dataframe.replace()](https://www.geeksforgeeks.org/python-pandas-dataframe-replace/)<br>
[12][Shane Lynn - Delete Rows & Columns in DataFrames Quickly using Pandas Drop](https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/)<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>
[][]()<br>

***
# End